In [62]:
import polars as pl
import seaborn as sns
from os import path
import requests
from bs4 import BeautifulSoup as bs
import re

In [63]:
team_stats = path.join("..", "data", "external", "Team stats.xlsx")

In [64]:
stats_df = pl.read_excel(team_stats, sheet_name="stats_clean", engine="xlsx2csv")

In [65]:
print(stats_df)

shape: (19, 53)
┌──────┬──────────────────────────┬──────┬──────┬───┬─────────────┬──────┬──────────┬──────────┐
│ RNK  ┆ TEAM                     ┆ GP   ┆ W    ┆ … ┆ DF 100 Poss ┆ BLK% ┆ eFG%     ┆ TS%      │
│ ---  ┆ ---                      ┆ ---  ┆ ---  ┆   ┆ ---         ┆ ---  ┆ ---      ┆ ---      │
│ i64  ┆ str                      ┆ i64  ┆ i64  ┆   ┆ f64         ┆ f64  ┆ f64      ┆ f64      │
╞══════╪══════════════════════════╪══════╪══════╪═══╪═════════════╪══════╪══════════╪══════════╡
│ 1    ┆ ALBA Berlin              ┆ 34   ┆ 14   ┆ … ┆ 28.5        ┆ 5.5  ┆ 0.527565 ┆ 0.560578 │
│ 2    ┆ Anadolu Efes Istanbul    ┆ 40   ┆ 23   ┆ … ┆ 27.7        ┆ 7.5  ┆ 0.558292 ┆ 0.596395 │
│ 3    ┆ AS Monaco                ┆ 38   ┆ 21   ┆ … ┆ 28.2        ┆ 5.9  ┆ 0.548507 ┆ 0.582909 │
│ 4    ┆ AX Armani Exchange Milan ┆ 36   ┆ 22   ┆ … ┆ 26.8        ┆ 6.8  ┆ 0.517345 ┆ 0.548064 │
│ …    ┆ …                        ┆ …    ┆ …    ┆ … ┆ …           ┆ …    ┆ …        ┆ …        │
│ 16   ┆ UNICS

In [66]:
def get_game_links():
    base_url = "https://www.euroleaguebasketball.net/euroleague/game-center/?round=1&season=E2022"
    site = requests.get(base_url)
    print(site.status_code)
    soup = bs(str(site.content), 'html.parser')

    links = []

    for link in soup.find_all("a"):
        links.append(link.get("href"))

    link_format = r"/euroleague/game-center/[\w-]+/[\w-]+/E2023/\d\d/"
    r = re.compile(link_format)
    newlist = list(filter(r.match, links))

    return list(dict.fromkeys(newlist))

In [67]:
res = get_game_links()

200


In [68]:
print(len(res))
print(res)

['/euroleague/game-center/2023-24/partizan-mozzart-bet-belgrade-fc-barcelona/E2023/22/', '/euroleague/game-center/2023-24/fenerbahce-beko-istanbul-panathinaikos-athens/E2023/21/', '/euroleague/game-center/2023-24/anadolu-efes-istanbul-ldlc-asvel-villeurbanne/E2023/19/', '/euroleague/game-center/2023-24/baskonia-vitoria-gasteiz-fc-bayern-munich/E2023/24/', '/euroleague/game-center/2023-24/real-madrid-zalgiris-kaunas/E2023/23/', '/euroleague/game-center/2023-24/ea7-emporio-armani-milan-olympiacos-piraeus/E2023/20/', '/euroleague/game-center/2023-24/crvena-zvezda-meridianbet-belgrade-as-monaco/E2023/25/', '/euroleague/game-center/2023-24/virtus-segafredo-bologna-alba-berlin/E2023/27/', '/euroleague/game-center/2023-24/valencia-basket-maccabi-playtika-tel-aviv/E2023/26/']
